### My wrapper for Light Game

In [1]:
!pip install gym

In [2]:
import gym.spaces
from gym.spaces import Discrete, Box

C:\Users\Tim\Anaconda3\envs\gpu2\lib\site-packages\gym\core.py:26: UserWarning: WARN: Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+
  "Gym minimally supports python 3.6 as the python foundation not longer supports the version, please update your version to 3.7+"


In [1]:
import gym
import random, math
import numpy as np
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

        
from LightEnvCopy import LightEnv
import gym.spaces
from gym.spaces import Discrete, Box

from ray.rllib.env.env_context import EnvContext
from ray.rllib.models import ModelCatalog

from collections import namedtuple

# Do the math to figure out our screen dimensions
SCREEN_WIDTH = 800
SCREEN_HEIGHT = 600
SCREEN_TITLE = "Game 1: Let There Be Light!"

SPRITE_SCALING = 0.25

# How fast the camera pans to the player. 1.0 is instant.
CAMERA_SPEED = 0.1

PLAYER_MOVEMENT_SPEED = 7
BOMB_COUNT = 5
TORCH_COUNT = 1
PLAYING_FIELD_WIDTH = 800 #1600
PLAYING_FIELD_HEIGHT = 600 #1600
REWARD_COUNT = 1 #TF - Add in reward
END_GAME = False
torch_collected = False

#TF Start - Adding in actions for action conversion

# COnvenient data structure to hold information about actions
Action = namedtuple('Action', 'name index delta_i delta_j')

up = Action('up', 0, -1, 0)    
down = Action('down', 1, 1, 0)    
left = Action('left', 2, 0, -1)    
right = Action('right', 3, 0, 1)    

index_to_actions = {}
for action in [up, down, left, right]:
    index_to_actions[action.index] = action
# print(index_to_actions[0].name)
str_to_actions = {}
for action in [up, down, left, right]:
    str_to_actions[action.name] = action
#TF End - Adding in actions for action conversion

    

class LightEnvWrapper(gym.Env, LightEnv):
    """Class that wraps the Lights Environment to make it 
    compatible with RLLib."""

    metadata = {"render.modes": ["rgb_array", "state_pixels"]}
    
    def __init__(self, config: EnvContext):
        super().__init__(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        game_size = config.get("size_env")

        self.mygame = LightEnv
        #The action space is a choice of 9 actions: U/D/L/R/UR/DR/DL/UL/DO NOTHING. Not continuous
        #because speed of agent is fixed. Or potentially just 4: U/D/L/R.
        self.action_space = Discrete(4)
        #The observation space is continuous, as there is a varying amount to be seen based on the
        #proximity to the light source, depth of shadow etc. Not sure what to say though.. <<HELP>>.
        #I guess consequences of actions is either just 0, -1 or +100?
        self.observation_space = Box(0, 255, shape=(600,800,3), dtype=np.uint8)
        
#         To resize the image use skimage. Image will be of the full env. Can do partial observability later to test, if time.
#         image = the image path
#         image_resized = resize(image, (image.shape[0] // 10, image.shape[1] // 10),
#                                anti_aliasing=True)
        

    def reset(self):
        #This method resets the state of the environment for a new episode and also returns an 
        #initial observation.
        obs_mygame = self.mygame.reset(self) #this won't work until included obs in main script
        print("obs_mygame: ", obs_mygame)
#         obs = self.convert_observations(obs_mygame)
        return obs_mygame

    def step(self, action):
        
        assert action in [0, 1, 2, 3] #0-up,1-down,2-left,3-right.
        
        #Things to do:
        # actions for my env = self.convert(actions from rllib)
        # MyEnvObs, MyEnvReward, done = self.mygame.step(actions for my env)
        # Convert the environment observations into a form readable by rllib:
        # obs = self.convert_observations(MyEnvObs)
        actions_myenv = index_to_actions[action].name #returns a word, one of: up/down/left/right
        
        obs, reward, done = self.mygame.step(actions_myenv)

        #Update window
        window = arcade.get_window()
        
        #Compute observation extracted from window (rescaled /10: 80x60)
        obs_mygame = window.resize(window, (window.shape[0] // 10, window.shape[1] // 10),anti_aliasing=True)
        print(obs_mygame)
        # Run the game with actions passed. So something like 
        # window = MyGameExplodingBombs(SCREEN_WIDTH, SCREEN_HEIGHT, SCREEN_TITLE)
        # arcade.run()
    
#         obs = self.convert_observations(obs_mygame)

        return obs_mygame, reward, done, {}

    def seed(self, seed=None):
        random.seed(seed)

    def convert_observations(self, obs_mygame): #Not needed? This is just for rescaling?
        # We normalize and concatenate observations

        relative_coord = obs_mygame['relative_coordinates']/self.mygame.size
        surroundings = obs_mygame['surroundings'] / 4
        
        obs = np.concatenate([relative_coord, surroundings.flatten()])

        return obs
    def render(self, mode='state_pixels'):
        self.mygame.render(mode)
        


pygame 2.1.2 (SDL 2.0.18, Python 3.6.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


### Now run the rllib script to train the agent

In [2]:
# !pip install ray==1.11.0
import gym
import ray.rllib.agents.ppo.ppo as ppo


In [3]:
import ray
import ray.rllib.agents.dqn as dqn
from ray.tune.logger import pretty_print

config = dqn.DEFAULT_CONFIG.copy()
config["framework"] = "torch"
config["dueling"] = False
config["double_q"] = False
config["env"] = LightEnvWrapper
config["env_config"] = { "size_env": 15}
config["model"] = { "fcnet_hiddens": [64, 64],
                    "fcnet_activation": "relu",
    }

trainer = dqn.DQNTrainer(config=config)

# Can optionally call trainer.restore(path) to load a checkpoint.

avg_rewards = []

for i in range(100):
    # Perform one iteration of training the policy with PPO
    result = trainer.train()
    #print(pretty_print(result))
    print(result['episode_reward_mean'])
    avg_rewards.append(result['episode_reward_mean'])

    if i % 10 == 0:
        checkpoint = trainer.save()
        print("checkpoint saved at", checkpoint)


2022-08-25 12:10:45,257	WARNING rollout_worker.py:499 -- We've added a module for checking environments that are used in experiments. It will cause your environment to fail if your environment is not set upcorrectly. You can disable check env by setting `disable_env_checking` to True in your experiment config dictionary. You can run the environment checking module standalone by calling ray.rllib.utils.check_env(env).
2022-08-25 12:10:45,258	WARNING env.py:121 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.


Shader 1 loading


C:\Users\Tim\Anaconda3\envs\gpu2\lib\site-packages\gym\spaces\box.py:142: UserWarning: WARN: Casting input x to numpy array.
  logger.warn("Casting input x to numpy array.")


Shader 1 loading
obs_mygame:  <LightEnvWrapper instance>


TypeError: int() argument must be a string, a bytes-like object or a number, not 'LightEnvWrapper'

### Example custom wrapper from ray documentation

In [ ]:
class SimpleCorridor(gym.Env):
    """Example of a custom env in which you have to walk down a corridor.
    You can configure the length of the corridor via the env config."""

    def __init__(self, config: EnvContext):
        self.end_pos = config["corridor_length"]
        self.cur_pos = 0
        self.action_space = Discrete(2)
        self.observation_space = Box(0.0, self.end_pos, shape=(1,), dtype=np.float32)
        # Set the seed. This is only used for the final (reach goal) reward.
        self.seed(config.worker_index * config.num_workers)

    def reset(self):
        self.cur_pos = 0
        return [self.cur_pos]

    def step(self, action):
        assert action in [0, 1], action
        if action == 0 and self.cur_pos > 0:
            self.cur_pos -= 1
        elif action == 1:
            self.cur_pos += 1
        done = self.cur_pos >= self.end_pos
        # Produce a random reward when we reach the goal.
        return [self.cur_pos], random.random() * 2 if done else -0.1, done, {}

    def seed(self, seed=None):
        random.seed(seed)
